In [ ]:
import re
import time
import os
import openai
import anthropic
import backoff
import pandas as pd
from itertools import product

# Story Geneartion

### Char Information

In [2]:
# Char Information 

CHAR1_NAME = ['John']
CHAR1_PRONOUN = ['He']
CHAR1_PRONOUN_LOW = ['he']
CHAR1_PROSESSIVE = ['his']
CHAR1_SELF = ['him']

CHAR2_NAME = ['Sam']
CHAR2_PRONOUN = ['He']
CHAR2_SELF = ['him']

char = {
    'john': {
        'name' : 'John',
        'pronoun' : 'He',
        'pronoun_lower' : 'he',
        'possessive' : 'his',
        'self' : 'him'
    },
    'sam' : {
        'name' : 'Sam',
        'pronoun' : 'He',
        'pronoun_lower' : 'he',
        'possessive' : 'his',
        'self' : 'him'
    },
    'mary' : {
        'name' : 'Mary',
        'pronoun' : 'She',
        'pronoun_lower' : 'she',
        'possessive' : 'her',
        'self' : 'her'
    },
    'sarah' : {
        'name' : 'Sarah',
        'pronoun' : 'She',
        'pronoun_lower' : 'she',
        'possessive' : 'her',
        'self' : 'her'
    }
}

### Competition Info

In [3]:
# JSON with level version 
# All information must be a sentence. (end with period)

# -- Character  -- #
# There will be at most 2 characters in a given story. 
# Each character can include 1) explicit importance of goals, 2) expectation (both implicit and explicit), and 3) controllability (explicit and char1 only) 

## First character: 
## char1 is a main character. The focus of the story and can change the situation (high control).

# Pottery info

pottery_info = {
    # Char 1
    'char1' : { 
        # Context 
        'context':{
            'low': "{char1:name} is participanting in a practice section for a pottery competition in which {char1:pronoun_lower} has to produce 100 potteries in 3 hours.",
            'medium':"{char1:name} is participanting in a pottery competition, in which {char1:pronoun_lower} would win $100 if {char1:pronoun_lower} can produce 100 potteries in 3 hours.",
            'high':"{char1:name} is participanting in a pottery competition, in which {char1:pronoun_lower} would win $10000 if {char1:pronoun_lower} can produce 100 potteries in 3 hours."
        },
        # Importance
        'importance':{
            'low': "{char1:pronoun} cares little about this competition.",
            'high': "{char1:pronoun} cares a lot about this competition."
        },
        # Expectation
        'expectation':{
            'low': "{char1:pronoun} expects to to do poorly.",
            'high': "{char1:pronoun} expects to to do well."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char1:name}'s thoughts are on {char1:self}self.",
            'event': "{char1:name}'s thoughts are on the outcome of the competition.",
            'other': "{char1:name}'s thoughts are on {char2:name}."
        }
    },
    # Char 2
    'char2' : { 
        # Context
        'relationship':{
            'close': "{char2:name} is a mentor of {char1:name}.",
            'far': "{char2:name} is a younger cousin of {char1:name}."
        },
        # Responsibility
        'responsibility':{
            'hinder':"{char2:name} has been hindering {char1:name} from time to time for this competition.",
            'help':"{char2:name} has been helping {char1:name} from time to time for this competition."
        },
        # importance
        'importance':{
            'low':"{char2:name} cares a little about this competition.",
            'high':"{char2:name} cares a lot about this competition."
        },
        # Expectation
        'expectation':{
            'low':"{char2:name} expects {char1:name} to do poorly.",
            'high':"{char2:name} expects {char1:name} to do well."
        },
        # Awareness
        'aware':{
            'aware':"{char1:name} is aware of {char2:name}'s thoughts and expectation toward this competition.",
            'not-aware':"{char1:name} is not aware of {char2:name}'s thoughts and expectation toward this competition."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char2:name}'s thoughts are on {char2:self}self.",
            'event': "{char2:name}'s thoughts are on the outcome of the competition.",
            'other': "{char2:name}'s thoughts are on {char1:name}."
        }
    },
    # Event
    'event': {
        # Context
        'intro': {
            'begin': "The competition begins."
        },
        # Progress (All these progresses are independent.)
        'progress': {
            'early-below-exp' : "The clock rangs after the first hour has passed, {char1:name} has finished 25 potteries.", 
            'early-above-exp' : "The clock rangs after the first hour has passed, {char1:name} has finished 40 potteries.",
            'late-below-exp'  : "After some time has passed, the announcer announces that 30 minutes remains, {char1:name} has finished 75 potteries.",
            'late-above-exp'  : "After some time has passed, the announcer announces that 30 minutes remains, {char1:name} has finished 90 potteries.", 
            'finish-not-done': "When the time is up, {char1:name} finished 80 potteries.",
            'finish-done': "When the time is up, {char1:name} finished 100 potteries."
        }
    }
}

pottery_story_char1 = ["char1:context", "char1:importance", "char1:expectation", 
                       "char2:relationship", "char2:responsibility", "char2:importance", "char2:expectation", "char2:aware",
                       "event:intro", "event:progress", "char1:attention"]

### Project Story

In [4]:
# Project info

project_info = {
    # Char 1
    'char1' : { 
        # Context 
        'context':{
            'low': "{char1:name} is a student working on a small classroom presentation.",
            'medium':"{char1:name} is a student working on a science project for a school competition.",
            'high':"{char1:name} is a writer working on his book publication."
        },
        # Importance
        'importance':{
            'low': "{char1:pronoun} cares little about this work.",
            'high': "{char1:pronoun} cares a lot about this work."
        },
        # Expectation
        'expectation':{
            'low': "{char1:pronoun} expects to to do poorly.",
            'high': "{char1:pronoun} expects to to do well."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char1:name}'s thoughts are on {char1:self}self.",
            'event': "{char1:name}'s thoughts are on the outcome of the competition.",
            'other': "{char1:name}'s thoughts are on {char2:name}."
        }
    },
    # Char 2
    'char2' : { 
        # Context
        'relationship':{
            'close': "{char2:name} is a mentor of {char1:name}.",
            'far': "{char2:name} is a younger cousin of {char1:name}."
        },
        # Responsibility
        'responsibility':{
            'hinder':"{char2:name} has been hindering {char1:name} from time to time for this competition.",
            'help':"{char2:name} has been helping {char1:name} from time to time for this competition."
        },
        # importance
        'importance':{
            'low':"{char2:name} cares a little about this work.",
            'high':"{char2:name} cares a lot about this work."
        },
        # Expectation
        'expectation':{
            'low':"{char2:name} expects {char1:name} to do poorly.",
            'high':"{char2:name} expects {char1:name} to do well."
        },
        # Awareness
        'aware':{
            'aware':"{char1:name} is aware of {char2:name}'s thoughts and expectation toward this work.",
            'not-aware':"{char1:name} is not aware of {char2:name}'s thoughts and expectation toward this work."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char2:name}'s thoughts are on {char2:self}self.",
            'event': "{char2:name}'s thoughts are on the outcome of the project.",
            'other': "{char2:name}'s thoughts are on {char1:name}."
        }
    },
    # Event
    'event': {
        # Context
        'intro': {
            'begin': "The work is due in 2 months."
        },
        # Progress (All these progresses are independent.)
        'progress': {
            'early-below-exp' : "After three weeks, {char1:name} has to report the progress and {char1:pronoun_lower} has finished around 25% of the work. {char2:name} also is aware of this.", 
            'early-above-exp' : "After three weeks, {char1:name} has to report the progress and {char1:pronoun_lower} has finished around 40% of the work. {char2:name} also is aware of this.",
            'late-below-exp'  : "One week before the deadline {char1:name} has to report the progress, {char1:pronoun_lower} has finished around 75% of the work. {char2:name} also is aware of this.",
            'late-above-exp'  : "One week before the deadline {char1:name} has to report the progress, {char1:pronoun_lower} has finished around 90% of the work. {char2:name} also is aware of this.", 
            'finish-not-done': "When the time is up, {char1:name} finished 80% of the work. {char2:name} also is aware of this.",
            'finish-done': "When the time is up, {char1:name} finished 100% of the work. {char2:name} also is aware of this."
        }
    }
}

project_story_char1 = ["char1:context", "char1:importance", "char1:expectation", 
                       "char2:relationship", "char2:responsibility", "char2:importance", "char2:expectation", "char2:aware",
                       "event:intro", "event:progress", "char1:attention"]

### Sport (Basketball) Event 

In [6]:
# Basketball info

basketball_info = {
    # Char 1
    'char1' : { 
        # Context 
        'context':{
            'low': "{char1:name} is participating in a friendly basketball match with friends.",
            'medium':"{char1:name} is participating in a college basketball weekly match.",
            'high':"{char1:name} is participating in the deciding match of the NBA finals."
        },
        # Importance
        'importance':{
            'low': "{char1:pronoun} cares little about this match.",
            'high': "{char1:pronoun} cares a lot about this match."
        },
        # Expectation
        'expectation':{
            'low': "{char1:pronoun} expects to to do poorly.",
            'high': "{char1:pronoun} expects to to do well."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char1:name}'s thoughts are on {char1:self}self.",
            'event': "{char1:name}'s thoughts are on the outcome of the match.",
            'other': "{char1:name}'s thoughts are on {char2:name}."
        }
    },
    # Char 2
    'char2' : { 
        # Context
        'relationship':{
            'close': "{char2:name} is a coach of {char1:name}.",
            'far': "{char2:name} is a younger cousin of {char1:name}."
        },
        # Responsibility
        'responsibility':{
            'hinder':"{char2:name} has been hindering {char1:name} from time to time for today's match.",
            'help':"{char2:name} has been helping {char1:name} from time to time for today's match."
        },
        # importance
        'importance':{
            'low':"{char2:name} cares a little about this match.",
            'high':"{char2:name} cares a lot about this match."
        },
        # Expectation
        'expectation':{
            'low':"{char2:name} expects {char1:name} to do poorly.",
            'high':"{char2:name} expects {char1:name} to do well."
        },
        # Awareness
        'aware':{
            'aware':"{char1:name} is aware of {char2:name}'s thoughts and expectation toward this match.",
            'not-aware':"{char1:name} is not aware of {char2:name}'s thoughts and expectation toward this match."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char2:name}'s thoughts are on {char2:self}self.",
            'event': "{char2:name}'s thoughts are on the outcome of the match.",
            'other': "{char2:name}'s thoughts are on {char1:name}."
        }
    },
    # Event
    'event': {
        # Context
        'intro': {
            'begin': "The match begins."
        },
        # Progress (All these progresses are independent.)
        'progress': {
            'early-below-exp' : "At the end of the first quarter, The score is 10-15, {char1:name}'s team to the other team.", 
            'early-above-exp' : "At the end of the first quarter, The score is 20-14, {char1:name}'s team to the other team.",
            'late-below-exp'  : "At the end of the third quarter, The score is 50-60, {char1:name}'s team to the other team.",
            'late-above-exp'  : "At the end of the third quarter, The score is 62-51, {char1:name}'s team to the other team.", 
            'finish-not-done': "The game ends and the score is 84-95, {char1:name}'s team to the other team.",
            'finish-done': "The game ends and the score is 97-90, {char1:name}'s team to the other team."
        }
    }
}

basketball_story_char1 = ["char1:context", "char1:importance", "char1:expectation", 
                       "char2:relationship", "char2:responsibility", "char2:importance", "char2:expectation", "char2:aware",
                       "event:intro", "event:progress", "char1:attention"]

### Exam Story

In [7]:
# Exam info

exam_info = {
    # Char 1
    'char1' : { 
        # Context 
        'context':{
            'low': "Today is the day of the mock SAT university exam. {char1:name} is a student taking today's exam.",
            'medium':"Today is the day of the final exam for a university class that counts for 30% of the grade. {char1:name} is a student taking today's exam.",
            'high':"Today is the day of the SAT exam. {char1:name} is a student taking today's exam."
        },
        # Importance
        'importance':{
            'low': "The exam is not very important to {char1:name}.",
            'high': "The exam is very important to {char1:name}."
        },
        # Expectation
        'expectation':{
            'low': "{char1:pronoun} expects to to do poorly.",
            'high': "{char1:pronoun} expects to to do well."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char1:name}'s thoughts are on {char1:self}self.",
            'event': "{char1:name}'s thoughts are on the exam.",
            'other': "{char1:name}'s thoughts are on {char2:name}."
        }
    },
    # Char 2
    'char2' : { 
        # Context
        'relationship':{
            'close': "{char2:name} is a tutor of {char1:name}.",
            'far': "{char2:name} is a younger cousin of {char1:name}."
        },
        # Responsibility
        'responsibility':{
            'hinder':"{char2:name} has been hindering {char1:name} from time to time in preparing for the exam.",
            'help':"{char2:name} has been helping {char1:name} from time to time in preparing for the exam."
        },
        # importance
        'importance':{
            'low':"It is not very important to {char2:name} that {char1:name} does well in the exam.",
            'high':"It is very important to {char2:name} that {char1:name} does well in the exam."
        },
        # Expectation
        'expectation':{
            'low':"{char2:name} expects {char1:name} to do poorly.",
            'high':"{char2:name} expects {char1:name} to do well."
        },
        # Awareness
        'aware':{
            'aware':"{char1:name} is aware of {char2:name}'s thoughts and expectation toward this exam.",
            'not-aware':"{char1:name} is not aware of {char2:name}'s thoughts and expectation toward this exam."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char2:name}'s thoughts are on {char2:self}self.",
            'event': "{char2:name}'s thoughts are on the exam.",
            'other': "{char2:name}'s thoughts are on {char1:name}."
        }
    },
    # Event
    'event': {
        # Context
        'intro': {
            'begin': "The exam begins. It lasts for three hours."
        },
        # Progress (All these progresses are independent.)
        'progress': {
            'early-below-exp' : "The clock rings, indicating that one hour has passed. {char1:name} has finish around 15% of the exam.", 
            'early-above-exp' : "The clock rings, indicating that one hour has passed. {char1:name} has finish around 35% of the exam.",
            'late-below-exp'  : "The announcement announces that only thirty minutes remain. {char1:name} has finish around 65% of the exam.",
            'late-above-exp'  : "The announcement announces that only thirty minutes remain. {char1:name} has finish around 85% of the exam.", 
            'finish-not-done': "When the time is up, {char1:name} finished 100% of the exam and believes that {char1:pronoun_lower} did poorly. {char1:name} tells {char2:name} about what he did.",
            'finish-done': "When the time is up, {char1:name} finished 100% of the exam and believes that {char1:pronoun_lower} did well. {char1:name} tells {char2:name} about what he did."
        }
    }
}

exam_story_char1 = ["char1:context", "char1:importance", "char1:expectation", 
                       "char2:relationship", "char2:responsibility", "char2:importance", "char2:expectation", "char2:aware",
                       "event:intro", "event:progress", "char1:attention"]

### Social Event Ver 1

In [8]:
# Social event ver 1 info
# Exam info

social1_info = {
    # Char 1
    'char1' : { 
        # Context 
        'context':{
            'low': "{char1:name} is friends with {char2:name} and is going on a trip with {char2:self}.",
            'medium':"{char1:name} is good friends with {char2:name} and is going on a trip with {char2:self}.",
            'high':"{char1:name} is best friends with {char2:name} and is gooing on a trip with {char2:self}."
        },
        # Importance
        'importance':{
            'low': "{char1:name} cares a little about this friendship and wants it to become better.",
            'high': "{char1:name} cares a lot about this friendship and wants it to become better."
        },
        # Expectation
        'expectation':{
            'low': "{char1:pronoun} expects their friendship to stay about the same as a result of the trip.",
            'high': "{char1:pronoun} expects their friendship to become deeper as a result of the trip."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char1:name}'s thoughts are on {char1:self}self.",
            'event': "{char1:name}'s thoughts are on the friendship.",
            'other': "{char1:name}'s thoughts are on {char2:name}."
        }
    },
    # Char 2
    'char2' : { 
        # No relationship since it's alrady in Char 1 context
        # Responsibility
        'responsibility':{
            'hinder':"{char2:name} has not made an effort to make time for their friendship.",
            'help':"{char2:name} has made an effort to make time for their friendship."
        },
        # importance
        'importance':{
            'low':"{char2:name} cares a little about the friendship.",
            'high':"{char2:name} cares a lot about the friendship."
        },
        # Expectation
        'expectation':{
            'low':"{char2:name} expects {char1:name} to put in a small amount of effort on their friendship and the trip.",
            'high':"{char2:name} expects {char1:name} to put in a big effort on their friendship and the trip."
        },
        # Awareness
        'aware':{
            'aware':"{char1:name} is aware of {char2:name}'s thoughts and expectation around their friendship.",
            'not-aware':"{char1:name} is not aware of {char2:name}'s thoughts and expectation around their friendship."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char2:name}'s thoughts are on {char2:self}self.",
            'event': "{char2:name}'s thoughts are on the friendship.",
            'other': "{char2:name}'s thoughts are on {char1:name}."
        }
    },
    # Event
    'event': {
        # Context
        'intro': {
            'begin': "The trip is three days long."
        },
        # Progress (All these progresses are independent.)
        'progress': {
            'early-below-exp' : "After the first day, {char1:name} checks in with {char1:self}self and finds that {char1:pronoun_lower} has put in a little effort to improve their friendship, and {char1:pronoun_lower} feels that their friendship is about the same.", 
            'early-above-exp' : "After the first day, {char1:name} checks in with {char1:self}self and finds that {char1:pronoun_lower} has put in a little effort to improve their friendship, and {char1:pronoun_lower} feels that their friendship is a tiny bit better.",
            'late-below-exp'  : "In the morning of the third day, {char1:name} checks in with {char1:self}self and finds that {char1:pronoun_lower} has put in a lot of effort to improve their friendship, and {char1:pronoun_lower} feels that their friendship is not improving.",
            'late-above-exp'  : "In the morning of the third day, {char1:name} checks in with {char1:self}self and finds that {char1:pronoun_lower} has put in a lot of effort to improve their friendship, and {char1:pronoun_lower} feels that their friendship is a little bit better'.", 
            'finish-not-done': "After the trip, {char1:name} thinks {char1:pronoun_lower} has done {char1:possessive} best to improve the friendship, and {char1:pronoun_lower} feels that their friendship is not going to improve.",
            'finish-done': "After the trip, {char1:name} thinks {char1:pronoun_lower} has done {char1:possessive} best to improve the friendship, and {char1:pronoun_lower} feels that their friendship is deeper."
        }
    }
}

social1_story_char1 = ["char1:context", "char1:importance", "char1:expectation", 
                       "char2:responsibility", "char2:importance", "char2:expectation", "char2:aware",
                       "event:intro", "event:progress", "char1:attention"]

### Social Event Ver 2

In [9]:
# Social event ver 2 info

social2_info = {
    # Char 1
    'char1' : { 
        # Context 
        'context':{
            'low': "{char1:name} is going on a 3-day business trip with {char1:possessive} company. {char1:pronoun} has a plan to befriend a new employee, Steve.",
            'medium':"{char1:name} is going on a 3-day business trip with {char1:possessive} company. {char1:pronoun} has a plan to befriend a new boss, Steve.",
            'high':"{char1:name} is going on a 3-day business trip with {char1:possessive} company. {char1:pronoun} has a plan to befriend a new business partner, Steve."
        },
        # Importance
        'importance':{
            'low': "It is not very important for {char1:name} to achieve the plan.",
            'high': "It is very important for {char1:name} to achieve the plan."
        },
        # Expectation
        'expectation':{
            'low': "{char1:pronoun} expects to do poorly.",
            'high': "{char1:pronoun} expects to do well."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char1:name}'s thoughts are on {char1:self}self.",
            'event': "{char1:name}'s thoughts are on the relationship.",
            'other': "{char1:name}'s thoughts are on {char2:name}."
        }
    },
    # Char 2
    'char2' : { 
        # Relationship
        'relationship':{
            'close':"{char2:name} is a close colleague of {char1:name}.",
            'far':"{char2:name} is an acquaintance of {char1:name}."
        },
        # Responsibility
        'responsibility':{
            'hinder':"{char2:name} has been misleading {char1:name} on how to make new friends.",
            'help':"{char2:name} has been helping {char1:name} on how to make new friends."
        },
        # importance
        'importance':{
            'low':"{char2:name} cares a little about whether {char1:name} and Steve become friends.",
            'high':"{char2:name} cares a lot about whether {char1:name} and Steve become friends."
        },
        # Expectation
        'expectation':{
            'low':"{char2:name} expects {char1:name} to do poorly.",
            'high':"{char2:name} expects {char1:name} to do well."
        },
        # Awareness
        'aware':{
            'aware':"{char1:name} is aware of {char2:name}'s thoughts and expectation about the potential friendship.",
            'not-aware':"{char1:name} is not aware of {char2:name}'s thoughts and expectation about the potential friendship."
        },
        # Attention (after the outcome)
        'attention':{
            'self': "{char2:name}'s thoughts are on {char2:self}self.",
            'event': "{char2:name}'s thoughts are on the relationship of {char1:name} and Steve.",
            'other': "{char2:name}'s thoughts are on {char1:name}."
        }
    },
    # Event
    'event': {
        # Context
        'intro': {
            'begin': "The trip begins."
        },
        # Progress (All these progresses are independent.)
        'progress': {
            'early-below-exp' : "After the first day, {char1:name} got to say hello and briefly introduce {char1:self}self to Steve.", 
            'early-above-exp' : "After the first day, {char1:name} got to introduce {char1:self}self to Steve and have a small talk.",
            'late-below-exp'  : "After the morning of the third day, {char1:name} only got to introduce {char1:self}self to Steve and have a small talk.",
            'late-above-exp'  : "After the morning of the third day, {char1:name} got to introduce {char1:self}self to Steve and had a conversation about their backgrounds and interests.", 
            'finish-not-done': "After the trip, {char1:name} only got to introduce {char1:self}self to Steve and they had a couple of small talk.",
            'finish-done': "After the trip, {char1:name} had several conversations about their backgrounds and interests, as well as exchanged their contact information."
        }
    }
}

social2_story_char1 = ["char1:context", "char1:importance", "char1:expectation", 
                       "char2:relationship","char2:responsibility", "char2:importance", "char2:expectation", "char2:aware",
                       "event:intro", "event:progress", "char1:attention"]

### Function to create the dataset for all stories

In [10]:
# Helper function to access dict of arbitary keys 
# keys is a list of keys in order of accessing
def get_nested_value(dictionary, keys):
    if not keys:
        return dictionary
    key = keys[0]
    if key in dictionary:
        return get_nested_value(dictionary[key], keys[1:])
    else:
        return KeyError(f"Key '{key}' not found in the nested dictionary")  

# Function to generate all story as a DataFrame 
def gen_stories(template, info, char1, char2):
    '''
    - template is a list of string. <- This will also be columns name 
    - info is a dict.
    - char1 is a string (name) 
    - char2 is a string (name)
    '''
    # Get levels of all columns
    levels = {} 
    for col in template:
        levels[col] = list(get_nested_value(info, col.split(':')).keys())
    permutations = product(*levels.values())
    result = [dict(zip(levels.keys(), values)) for values in permutations]
    df = []
    # Loop through all patterns
    for perm in result:
        story = ''
        # Gen the story 
        for part in perm:
            keys = part.split(':') + [perm[part]]
            story += get_nested_value(info, keys) + ' '
        # Update the char info
        for k in char[char1]:
            story = story.replace('{char1:'+k+'}', char[char1][k])
            story = story.replace('{char2:'+k+'}', char[char2][k])
        # Save to data
        perm['char1_name'] = char1
        perm['char2_name'] = char2
        perm['story'] = story.strip()
        df.append(perm)
    
    return pd.DataFrame(df)

## Gen Stories

### Full stories

In [11]:
# Gen all stories

df_pottery = gen_stories(pottery_story_char1, pottery_info, 'john', 'sam')
df_pottery.to_csv('appraisal stories\\pottery_story_john_sam.csv')

df_project = gen_stories(project_story_char1, project_info, 'john','sam')
df_project.to_csv('appraisal stories\\project_story_john_sam.csv')

df_basketball = gen_stories(basketball_story_char1, basketball_info, 'john','sam')
df_basketball.to_csv('appraisal stories\\basketball_story_john_sam.csv')

df_exam = gen_stories(exam_story_char1, exam_info, 'john','sam')
df_exam.to_csv('appraisal stories\\exam_story_john_sam.csv')

df_social1 = gen_stories(social1_story_char1, social1_info, 'john','sam')
df_social1.to_csv('appraisal stories\\social1_story_john_sam.csv')

df_social2 = gen_stories(social2_story_char1, social2_info, 'john','sam')
df_social2.to_csv('appraisal stories\\social2_story_john_sam.csv')

### Human Test only stories
- plus 'early-below-exp' and 'early-above-exp'

In [12]:
df_pottery_human = df_pottery[(df_pottery['char1:attention'] == 'event') & 
                              (df_pottery['char1:context'] == 'medium') &
                              (df_pottery['char1:importance'] == 'high') &
                              (df_pottery['char2:responsibility'] == 'help') &
                              (df_pottery['char2:relationship'] == 'close') &
                              (df_pottery['char2:aware'] == 'aware') & 
                               df_pottery['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_project_human = df_project[(df_project['char1:attention'] == 'event') & 
                              (df_project['char1:context'] == 'medium') &
                              (df_project['char1:importance'] == 'high') &
                              (df_project['char2:responsibility'] == 'help') &
                              (df_project['char2:relationship'] == 'close') &
                              (df_project['char2:aware'] == 'aware') & 
                               df_project['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]


df_basketball_human = df_basketball[(df_basketball['char1:attention'] == 'event') & 
                                    (df_basketball['char1:context'] == 'medium') &
                                    (df_basketball['char1:importance'] == 'high') &
                                    (df_basketball['char2:responsibility'] == 'help') &
                                    (df_basketball['char2:relationship'] == 'close') &
                                    (df_basketball['char2:aware'] == 'aware') & 
                                    df_basketball['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_exam_human = df_exam[(df_exam['char1:attention'] == 'event') & 
                        (df_exam['char1:context'] == 'medium') &
                        (df_exam['char1:importance'] == 'high') &
                        (df_exam['char2:responsibility'] == 'help') &
                        (df_exam['char2:relationship'] == 'close') &
                        (df_exam['char2:aware'] == 'aware') & 
                         df_exam['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_social1_human = df_social1[(df_social1['char1:attention'] == 'event') & 
                        (df_social1['char1:context'] == 'medium') &
                        (df_social1['char1:importance'] == 'high') &
                        (df_social1['char2:responsibility'] == 'help') &
                        (df_social1['char2:aware'] == 'aware') & 
                         df_social1['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_social2_human = df_social2[(df_social2['char1:attention'] == 'event') & 
                        (df_social2['char1:context'] == 'medium') &
                        (df_social2['char1:importance'] == 'high') &
                        (df_social2['char2:responsibility'] == 'help') &
                        (df_social2['char2:relationship'] == 'close') &
                        (df_social2['char2:aware'] == 'aware') & 
                         df_social2['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_all = {'pottery': df_pottery_human,
          'project': df_project_human,
          'basketball': df_basketball_human,
          'exam': df_exam_human,
          'social1': df_social1_human,
          'social2': df_social2_human}

- 'not aware' set

In [13]:
df_pottery_human_not_aware = df_pottery[(df_pottery['char1:attention'] == 'event') & 
                              (df_pottery['char1:context'] == 'medium') &
                              (df_pottery['char1:importance'] == 'high') &
                              (df_pottery['char2:responsibility'] == 'help') &
                              (df_pottery['char2:relationship'] == 'close') &
                              (df_pottery['char2:aware'] == 'not-aware') & 
                               df_pottery['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_project_human_not_aware = df_project[(df_project['char1:attention'] == 'event') & 
                              (df_project['char1:context'] == 'medium') &
                              (df_project['char1:importance'] == 'high') &
                              (df_project['char2:responsibility'] == 'help') &
                              (df_project['char2:relationship'] == 'close') &
                              (df_project['char2:aware'] == 'not-aware') & 
                               df_project['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]


df_basketball_human_not_aware = df_basketball[(df_basketball['char1:attention'] == 'event') & 
                                    (df_basketball['char1:context'] == 'medium') &
                                    (df_basketball['char1:importance'] == 'high') &
                                    (df_basketball['char2:responsibility'] == 'help') &
                                    (df_basketball['char2:relationship'] == 'close') &
                                    (df_basketball['char2:aware'] == 'not-aware') & 
                                    df_basketball['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_exam_human_not_aware = df_exam[(df_exam['char1:attention'] == 'event') & 
                        (df_exam['char1:context'] == 'medium') &
                        (df_exam['char1:importance'] == 'high') &
                        (df_exam['char2:responsibility'] == 'help') &
                        (df_exam['char2:relationship'] == 'close') &
                        (df_exam['char2:aware'] == 'not-aware') & 
                         df_exam['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_social1_human_not_aware = df_social1[(df_social1['char1:attention'] == 'event') & 
                        (df_social1['char1:context'] == 'medium') &
                        (df_social1['char1:importance'] == 'high') &
                        (df_social1['char2:responsibility'] == 'help') &
                        (df_social1['char2:aware'] == 'not-aware') & 
                         df_social1['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_social2_human_not_aware = df_social2[(df_social2['char1:attention'] == 'event') & 
                        (df_social2['char1:context'] == 'medium') &
                        (df_social2['char1:importance'] == 'high') &
                        (df_social2['char2:responsibility'] == 'help') &
                        (df_social2['char2:relationship'] == 'close') &
                        (df_social2['char2:aware'] == 'not-aware') & 
                         df_social2['event:progress'].isin(['early-below-exp', 'early-above-exp', 'finish-not-done','finish-done'])]

df_all_not_aware = {'pottery': df_pottery_human_not_aware,
          'project': df_project_human_not_aware,
          'basketball': df_basketball_human_not_aware,
          'exam': df_exam_human_not_aware,
          'social1': df_social1_human_not_aware,
          'social2': df_social2_human_not_aware}

<br>

---

# LLM Generation

In [ ]:
# OpenAI API
client = openai.OpenAI()
FOURO = 'gpt-4o'
O1 = 'o1-2024-12-17'

@backoff.on_exception(backoff.expo, openai.RateLimitError, max_time=6000)
def chat_completions_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

def gptQuery(prompt, model = FOURO, temperature = 0, echo = False):
  out = chat_completions_with_backoff(model=model,
                                     messages=[{"role":"user","content":prompt}], 
                                     temperature=temperature, max_tokens = 200)
  if echo: print(out) 
  return out.choices[0].message.content

print(gptQuery(prompt= "Hello Test"))

In [16]:
# Anthropic API
client_anthropic = anthropic.Anthropic()
SONNET35 = "claude-3-5-sonnet-20240620"

def claudeQuery(prompt, model=SONNET35, temperature = 0, echo=False):
    out = client_anthropic.messages.create(
        model = model,
        max_tokens=200,
        temperature=temperature,
        system="",
        messages= [{"role": "user", "content": prompt}]
    )
    if echo: print(out.content[0].text)
    return out.content[0].text

print(claudeQuery(prompt = "Hello Test"))

Hello! How can I assist you today? Is there anything specific you'd like to ask or discuss?


In [ ]:
# LLaMA 3.1 405b and Gemini pro 1.5 from openrouter
GEMINIPRO = "google/gemini-pro-1.5"
LLAMA405 = "meta-llama/llama-3.1-405b-instruct"

or_client = openai.OpenAI(
  base_url = "https://openrouter.ai/api/v1",
  api_key = os.getenv("OPENROUTER_API_KEY"),
)

def openRouterQuery(prompt, model = GEMINIPRO, temperature = 0, echo=False):
    out = or_client.chat.completions.create(model=model,
                                     messages=[{"role":"user","content":prompt}], 
                                     temperature=temperature, max_tokens = 200)
    if echo: print(out.choices[0].message.content)
    try: 
      return(out.choices[0].message.content)
    except:
       return("")

print(openRouterQuery("Hello Test.", GEMINIPRO))
print(openRouterQuery("Hello Test.", LLAMA405))

In [18]:
models = [FOURO, SONNET35, GEMINIPRO, LLAMA405]

def generate(prompt, model, temperature=0, echo=False):
    if model == FOURO:
        return gptQuery(prompt, model, temperature, echo)
    if model == SONNET35:
        return claudeQuery(prompt, model, temperature, echo)
    if model in [GEMINIPRO, LLAMA405]:
        return openRouterQuery(prompt, model, temperature, echo)

In [20]:
# Prompt 
instruction = ("In this study, you will see a short story. " + 
               "The story will introduce a situation and two characters. "+
               "We want you to rate what emotions the character {char1_name} might be feeling at the end of the story. "+
               "We want you to rate the intensity of 20 possible emotions. "+
               'The rating ranges from "0 = no intensity (not felt)" to "6 = very high intensity".  '+
               "If you feel that the emotion is not present for the character, leave the intensity at 0.\n")

emotion_explain = ('When rating you can consider the emotion words provided as representing a large "emotion family" and may thus refer to a whole range of similar emotions. '+
                   'Thus, the Anger family also covers emotions such as rage, vexation, annoyance, indignation, fury, exasperation, or being cross or mad; the Fear family includes anxiety, worry, apprehensiveness, fright, or panic. '+
                   'Some of the words, such as love, hate, or guilt, can be used to refer to long-term affective states; but in this case rating those emotions as different from 0 means that you imagine the character to have a salient temporary feeling that belongs to the families of Love, Hate, or Guilt. ' +
                   'Once you have identified what emotions might be likely, choose the intensity you imagine the character might be feeling for that specific emotion. ' +
                   'Emotions are sometimes blended or mixed and may contain many different components. '+
                   'Please rate the intensity of ALL possible emotions you think the character might feel, whether they are unique emotions or blends, even if the intensities are different. '+
                   'So, for instance, if you imagine that the character might feel either Gratitude OR a combination of Anger and Hate, you should rate the imagined intensity for all three emotions even if you do not think that a blend of all three could happen.\n'
                   )

other_instruction = ('If the story takes place over a period of time, we want you to rate what emotions you think the character has at the end of the story and not emotions they may have felt during the time period. ' + 
                     'For instance, if the character was driving to meet a friend and the character shows up late, we want you to rate what their feelings might be when arriving late, and not what possible emotions that they may have felt while driving. ' +
                     'In the stories, there might be information which you consider to be irrelevant for how the character is feeling or information that you have access to, which the character does not. ' +
                     "This is by design. Please try to rate the emotions from the character's perspective based on the information in the stories\n")


emotion_words = 'Emotions: Interest, Amusement, Pride, Joy, Pleasure, Contentment, Love, Admiration, Relief, Compassion, Sadness, Guilt, Regret, Shame, Disappointment, Fear, Disgust, Contempt, Hate, Anger\n'
reversed_emotion_words = 'Emotions: Anger, Hate, Contempt, Disgust, Fear, Disappointment, Shame, Regret, Guilt, Sadness, Compassion, Relief, Admiration, Love, Contentment, Pleasure, Joy, Pride, Amusement, Interest\n'
# Random order 

story_prompt = 'Here is the story:\n\n{story}\n\n'

llm_instruction = 'Output format: Output ONLY numbers in order separated by commas without additional explantion and put them in <answer> tags.'
# emotion then numbers
llm_instruction_word = 'Output format: Output only the emotion words follows by the numbers in order separated by commas (e.g., Interest 0,Amusement 0,Pride 0) without additional explantion and put them in <answer> tags'



prompts = {
    # base instruction
    'base' : instruction + emotion_explain + other_instruction  + story_prompt + emotion_words+ llm_instruction,
    # Reserved order of emotion 
    'reversed_order': instruction + emotion_explain + other_instruction + story_prompt + reversed_emotion_words + llm_instruction,
    # Emotion word instruction
    'emotion_word': instruction + emotion_explain + other_instruction + story_prompt + emotion_words + llm_instruction_word,
    # Instruct to act as the character
    'act_as_char':"",
    # Instrct to predict how humans would response
    'predict_human':""
}

# function to run the experiment
# Return a Dataframe contain one more column for the response of each story
def run_experiment(stories, domain, prompt, model, temperature = 0, n = 1, aware="", echo=False):
    
    results = []
    for i in range(n):
        result = stories[domain].copy()
        # loop through all stories
        responses = []
        for story in result['story']:
            # Construction the prompt
            query = prompts[prompt].replace('{story}', story)
            response = generate(query, model, temperature)
            # Validate response but not more than 5 time:
            count = 0 
            while len(response.split(',')) != 20 and temperature !=0 and count <5:
                # Keep asking beside temperature = 0
                response = generate(query, model, temperature)
                count += 1
            responses.append(response)

            if echo:
                print(query)
                print(response)
                print( len(response.split(',')) )
        result['response'] = responses
        result['n'] = i
        results.append(result)

    # Write to file
    df = pd.concat(results)
    df['temperature'] = temperature 
    df['domain'] = domain
    df['model'] = model
    df['prompt'] = prompt
    # dealing with openrouter.ai name
    if model == GEMINIPRO:
        model = "gemini-pro-15"
    if model == LLAMA405:
        model = "LLAMA405"

    file_name = f'results\\{model}_{domain}_prompt={prompt}_temp={temperature}_{aware}.csv'
    df.to_csv(file_name, index=False)
    return df


## Code to run experiments 
- Break down per model and condition 
- Conditions:
  - temperature = 0. n = 1. All stories. Base Prompt only.
  - temperature = 0.5. n = 10. Only tested stories.
  - temperature = 1.0. n = 10. Only tested stories.

### Temperature = 0 case 

In [21]:
STORIES = ['pottery','project','basketball','exam','social1','social2']

In [ ]:
# GPT-4o


## Base Prompt 
run_experiment(df_all, STORIES[0], 'base', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[1], 'base', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[2], 'base', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[3], 'base', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[4], 'base', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[5], 'base', FOURO, temperature=0, n=1, echo=True)

## resersed order
run_experiment(df_all, STORIES[0], 'reversed_order', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[1], 'reversed_order', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[2], 'reversed_order', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[3], 'reversed_order', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[4], 'reversed_order', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[5], 'reversed_order', FOURO, temperature=0, n=1, echo=True)

## emotion words
run_experiment(df_all, STORIES[0], 'emotion_word', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[1], 'emotion_word', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[2], 'emotion_word', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[3], 'emotion_word', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[4], 'emotion_word', FOURO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[5], 'emotion_word', FOURO, temperature=0, n=1, echo=True)

In [ ]:
# Sonnet 3.5

## Base Prompt 
run_experiment(df_all, STORIES[0], 'base', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[1], 'base', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[2], 'base', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[3], 'base', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[4], 'base', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[5], 'base', SONNET35, temperature=0, n=1)

## resersed order
run_experiment(df_all, STORIES[0], 'reversed_order', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[1], 'reversed_order', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[2], 'reversed_order', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[3], 'reversed_order', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[4], 'reversed_order', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[5], 'reversed_order', SONNET35, temperature=0, n=1)

## emotion words
run_experiment(df_all, STORIES[0], 'emotion_word', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[1], 'emotion_word', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[2], 'emotion_word', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[3], 'emotion_word', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[4], 'emotion_word', SONNET35, temperature=0, n=1)
run_experiment(df_all, STORIES[5], 'emotion_word', SONNET35, temperature=0, n=1)


In [ ]:
# Gemini 1.5 pro

## Base Prompt 
run_experiment(df_all, STORIES[0], 'base', GEMINIPRO, temperature=0, n=1, echo=True)
run_experiment(df_all, STORIES[1], 'base', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[2], 'base', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[3], 'base', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[4], 'base', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[5], 'base', GEMINIPRO, temperature=0, n=1)

## resersed order
run_experiment(df_all, STORIES[0], 'reversed_order', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[1], 'reversed_order', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[2], 'reversed_order', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[3], 'reversed_order', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[4], 'reversed_order', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[5], 'reversed_order', GEMINIPRO, temperature=0, n=1)

## emotion words
run_experiment(df_all, STORIES[0], 'emotion_word', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[1], 'emotion_word', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[2], 'emotion_word', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[3], 'emotion_word', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[4], 'emotion_word', GEMINIPRO, temperature=0, n=1)
run_experiment(df_all, STORIES[5], 'emotion_word', GEMINIPRO, temperature=0, n=1)


In [ ]:
# LLaMA 3.1 405B

## Base Prompt 
run_experiment(df_all, STORIES[0], 'base', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[1], 'base', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[2], 'base', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[3], 'base', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[4], 'base', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[5], 'base', LLAMA405, temperature=0, n=1)


## resersed order
run_experiment(df_all, STORIES[0], 'reversed_order', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[1], 'reversed_order', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[2], 'reversed_order', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[3], 'reversed_order', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[4], 'reversed_order', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[5], 'reversed_order', LLAMA405, temperature=0, n=1)

## emotion words
run_experiment(df_all, STORIES[0], 'emotion_word', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[1], 'emotion_word', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[2], 'emotion_word', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[3], 'emotion_word', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[4], 'emotion_word', LLAMA405, temperature=0, n=1)
run_experiment(df_all, STORIES[5], 'emotion_word', LLAMA405, temperature=0, n=1)

### Temperature = 1 case

In [ ]:
# GPT-4o
run_experiment(df_all, STORIES[0], 'base', FOURO, temperature=1, n=10)
run_experiment(df_all, STORIES[1], 'base', FOURO, temperature=1, n=10)
run_experiment(df_all, STORIES[2], 'base', FOURO, temperature=1, n=10)
run_experiment(df_all, STORIES[3], 'base', FOURO, temperature=1, n=10)
run_experiment(df_all, STORIES[4], 'base', FOURO, temperature=1, n=10)
run_experiment(df_all, STORIES[5], 'base', FOURO, temperature=1, n=10)

In [ ]:
# Sonnet 3.5

run_experiment(df_all, STORIES[0], 'base', SONNET35, temperature=1, n=10)
run_experiment(df_all, STORIES[1], 'base', SONNET35, temperature=1, n=10)
run_experiment(df_all, STORIES[2], 'base', SONNET35, temperature=1, n=10)
run_experiment(df_all, STORIES[3], 'base', SONNET35, temperature=1, n=10)
run_experiment(df_all, STORIES[4], 'base', SONNET35, temperature=1, n=10)
run_experiment(df_all, STORIES[5], 'base', SONNET35, temperature=1, n=10)

In [ ]:
# Gemini 

run_experiment(df_all, STORIES[0], 'base', GEMINIPRO, temperature=1, n=10)
run_experiment(df_all, STORIES[1], 'base', GEMINIPRO, temperature=1, n=10)
run_experiment(df_all, STORIES[2], 'base', GEMINIPRO, temperature=1, n=10)
run_experiment(df_all, STORIES[3], 'base', GEMINIPRO, temperature=1, n=10)
run_experiment(df_all, STORIES[4], 'base', GEMINIPRO, temperature=1, n=10)
run_experiment(df_all, STORIES[5], 'base', GEMINIPRO, temperature=1, n=10)

In [ ]:
# LLama
run_experiment(df_all, STORIES[0], 'base', LLAMA405, temperature=1, n=10)
run_experiment(df_all, STORIES[1], 'base', LLAMA405, temperature=1, n=10)
run_experiment(df_all, STORIES[2], 'base', LLAMA405, temperature=1, n=10)
run_experiment(df_all, STORIES[3], 'base', LLAMA405, temperature=1, n=10)
run_experiment(df_all, STORIES[4], 'base', LLAMA405, temperature=1, n=10)
run_experiment(df_all, STORIES[5], 'base', LLAMA405, temperature=1, n=10)

## Not-Aware Case 
- I forget.

### Temperature = 0

In [ ]:
# Rerun basketball
run_experiment(df_all_not_aware, STORIES[2], 'base', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'reversed_order', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'emotion_word', FOURO, temperature=0, n=1, aware="not_aware")


run_experiment(df_all_not_aware, STORIES[2], 'base', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'reversed_order', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'emotion_word', SONNET35, temperature=0, n=1, aware="not_aware")


run_experiment(df_all_not_aware, STORIES[2], 'base', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'reversed_order', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'emotion_word', GEMINIPRO, temperature=0, n=1, aware="not_aware")

run_experiment(df_all_not_aware, STORIES[2], 'base', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'reversed_order', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'emotion_word', LLAMA405, temperature=0, n=1, aware="not_aware")


In [ ]:
# GPT-4o

## Base Prompt 
run_experiment(df_all_not_aware, STORIES[0], 'base', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'base', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'base', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'base', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'base', FOURO, temperature=0, n=1, aware="not_aware")

## resersed order
run_experiment(df_all_not_aware, STORIES[0], 'reversed_order', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'reversed_order', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'reversed_order', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'reversed_order', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'reversed_order', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'reversed_order', FOURO, temperature=0, n=1, aware="not_aware")

## emotion words
run_experiment(df_all_not_aware, STORIES[0], 'emotion_word', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'emotion_word', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'emotion_word', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'emotion_word', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'emotion_word', FOURO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'emotion_word', FOURO, temperature=0, n=1, aware="not_aware")

In [ ]:
# SONNET

## Base Prompt 
run_experiment(df_all_not_aware, STORIES[0], 'base', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'base', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'base', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'base', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'base', SONNET35, temperature=0, n=1, aware="not_aware")

## resersed order
run_experiment(df_all_not_aware, STORIES[0], 'reversed_order', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'reversed_order', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'reversed_order', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'reversed_order', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'reversed_order', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'reversed_order', SONNET35, temperature=0, n=1, aware="not_aware")

## emotion words
run_experiment(df_all_not_aware, STORIES[0], 'emotion_word', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'emotion_word', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'emotion_word', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'emotion_word', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'emotion_word', SONNET35, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'emotion_word', SONNET35, temperature=0, n=1, aware="not_aware")

In [ ]:
# GEMINI

## Base Prompt 
run_experiment(df_all_not_aware, STORIES[0], 'base', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'base', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'base', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'base', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'base', GEMINIPRO, temperature=0, n=1, aware="not_aware")

## resersed order
run_experiment(df_all_not_aware, STORIES[0], 'reversed_order', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'reversed_order', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'reversed_order', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'reversed_order', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'reversed_order', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'reversed_order', GEMINIPRO, temperature=0, n=1, aware="not_aware")

## emotion words
run_experiment(df_all_not_aware, STORIES[0], 'emotion_word', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'emotion_word', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'emotion_word', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'emotion_word', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'emotion_word', GEMINIPRO, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'emotion_word', GEMINIPRO, temperature=0, n=1, aware="not_aware")

In [ ]:
# LLAMA

## Base Prompt 
run_experiment(df_all_not_aware, STORIES[0], 'base', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'base', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'base', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'base', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'base', LLAMA405, temperature=0, n=1, aware="not_aware")

## resersed order
run_experiment(df_all_not_aware, STORIES[0], 'reversed_order', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'reversed_order', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'reversed_order', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'reversed_order', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'reversed_order', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'reversed_order', LLAMA405, temperature=0, n=1, aware="not_aware")

## emotion words
run_experiment(df_all_not_aware, STORIES[0], 'emotion_word', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'emotion_word', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'emotion_word', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'emotion_word', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'emotion_word', LLAMA405, temperature=0, n=1, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'emotion_word', LLAMA405, temperature=0, n=1, aware="not_aware")

#### Temperature = 1

In [ ]:
#Rerun basketball 

run_experiment(df_all_not_aware, STORIES[2], 'base', FOURO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', SONNET35, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', GEMINIPRO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', LLAMA405, temperature=1, n=10, aware="not_aware")

In [ ]:
# GPT-4o

## Base Prompt 
run_experiment(df_all_not_aware, STORIES[0], 'base', FOURO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'base', FOURO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', FOURO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'base', FOURO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'base', FOURO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'base', FOURO, temperature=1, n=10, aware="not_aware")

In [ ]:
# Sonnet

## Base Prompt 
run_experiment(df_all_not_aware, STORIES[0], 'base', SONNET35, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'base', SONNET35, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', SONNET35, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'base', SONNET35, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'base', SONNET35, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'base', SONNET35, temperature=1, n=10, aware="not_aware")

In [ ]:
# GEMINI

## Base Prompt 
run_experiment(df_all_not_aware, STORIES[0], 'base', GEMINIPRO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'base', GEMINIPRO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', GEMINIPRO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'base', GEMINIPRO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'base', GEMINIPRO, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'base', GEMINIPRO, temperature=1, n=10, aware="not_aware")

In [ ]:
# LLAMA

## Base Prompt 
run_experiment(df_all_not_aware, STORIES[0], 'base', LLAMA405, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[1], 'base', LLAMA405, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[2], 'base', LLAMA405, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[3], 'base', LLAMA405, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[4], 'base', LLAMA405, temperature=1, n=10, aware="not_aware")
run_experiment(df_all_not_aware, STORIES[5], 'base', LLAMA405, temperature=1, n=10, aware="not_aware")